# Linear Regression That Actually Works - Features, Interactions, Diagnostics

<hr>

<center>
<div>
<img src="https://raw.githubusercontent.com/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/main/notebooks/figures/mgmt_474_ai_logo_02-modified.png" width="200"/>
</div>
</center>

# <center><a class="tocSkip"></center>
# <center>MGMT47400 Predictive Analytics</center>
# <center>Professor: Davi Moreira </center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/04_linear_features_diagnostics.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Fit and interpret linear regression in a pipeline
2. Create interaction/polynomial features responsibly
3. Diagnose underfit/overfit using validation results
4. Use residual analysis to spot nonlinearity and heteroskedasticity
5. Translate coefficients into business meaning (with caveats)

---

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
RANDOM_SEED = 474
np.random.seed(RANDOM_SEED)
print("✓ Setup complete!")

## 1. Load Data and Create Splits

In [ ]:
california = fetch_california_housing(as_frame=True)
df = california.frame
X = df.drop(columns=['MedHouseVal'])
y = df['MedHouseVal']

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.20, random_state=RANDOM_SEED)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=RANDOM_SEED)

print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)} (locked)")

## 2. Baseline Linear Model with Preprocessing

In [ ]:
# Simple linear regression pipeline
baseline_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

baseline_pipeline.fit(X_train, y_train)

train_score = baseline_pipeline.score(X_train, y_train)
val_score = baseline_pipeline.score(X_val, y_val)

print("=== BASELINE LINEAR MODEL ===")
print(f"Train R²: {train_score:.4f}")
print(f"Val R²: {val_score:.4f}")
print(f"Overfit gap: {train_score - val_score:.4f}")

## 3. Coefficient Interpretation

### Understanding Linear Regression Coefficients

**What coefficients tell you:**
- Direction of relationship (positive/negative)
- Relative importance (after scaling)
- Magnitude of effect (with caveats)

**Interpretation caveats:**
- Only valid if features are scaled consistently
- Assumes linear relationship
- Affected by multicollinearity
- Correlation ≠ causation

In [ ]:
# Extract coefficients
coefficients = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': baseline_pipeline.named_steps['regressor'].coef_,
    'Abs_Coefficient': np.abs(baseline_pipeline.named_steps['regressor'].coef_)
}).sort_values('Abs_Coefficient', ascending=False)

print("=== LINEAR REGRESSION COEFFICIENTS ===")
print(coefficients)

# Visualize
plt.figure(figsize=(10, 6))
plt.barh(coefficients['Feature'], coefficients['Coefficient'])
plt.xlabel('Coefficient Value (after scaling)')
plt.title('Feature Importance via Linear Regression Coefficients')
plt.axvline(x=0, color='red', linestyle='--')
plt.tight_layout()
plt.show()

print(f"\nIntercept: {baseline_pipeline.named_steps['regressor'].intercept_:.4f}")
print(f"\n💡 Positive coefficients increase the prediction")
print(f"💡 Negative coefficients decrease the prediction")

## 4. Feature Engineering: Interactions

### When to Add Interactions

Interactions capture **combined effects** of features:
- Income × Location might matter more than either alone
- Age × Size might have nonlinear effects

**Warning:** Interactions increase features exponentially!

In [ ]:
# Let's create a simple interaction manually first
# Example: interaction between MedInc and AveRooms
X_train_interact = X_train.copy()
X_val_interact = X_val.copy()

X_train_interact['MedInc_x_AveRooms'] = X_train['MedInc'] * X_train['AveRooms']
X_val_interact['MedInc_x_AveRooms'] = X_val['MedInc'] * X_val['AveRooms']

# Fit model with interaction
interact_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

interact_pipeline.fit(X_train_interact, y_train)

train_score_interact = interact_pipeline.score(X_train_interact, y_train)
val_score_interact = interact_pipeline.score(X_val_interact, y_val)

print("=== WITH INTERACTION FEATURE ===")
print(f"Train R²: {train_score_interact:.4f}")
print(f"Val R²: {val_score_interact:.4f}")
print(f"\nImprovement over baseline: {val_score_interact - val_score:.4f}")

## 📝 PAUSE-AND-DO Exercise 1 (10 minutes)

**Task:** Add an interaction or polynomial block and measure validation change.

**Instructions:**
1. Use `PolynomialFeatures` to create degree=2 features
2. Fit a new pipeline
3. Compare validation scores
4. Watch for overfitting!

---

In [ ]:
# YOUR CODE HERE: Add polynomial features

poly_pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

poly_pipeline.fit(X_train, y_train)

train_score_poly = poly_pipeline.score(X_train, y_train)
val_score_poly = poly_pipeline.score(X_val, y_val)

print("=== POLYNOMIAL FEATURES (degree=2) ===")
print(f"Train R²: {train_score_poly:.4f}")
print(f"Val R²: {val_score_poly:.4f}")
print(f"Overfit gap: {train_score_poly - val_score_poly:.4f}")

# Count features
n_features_original = X_train.shape[1]
n_features_poly = poly_pipeline.named_steps['poly'].transform(X_train[:1]).shape[1]
print(f"\nOriginal features: {n_features_original}")
print(f"After polynomial (degree=2): {n_features_poly}")
print(f"Feature explosion: {n_features_poly / n_features_original:.1f}x increase")

### YOUR ANALYSIS:

**Observation 1: Performance**  
[Did polynomial features improve validation score?]

**Observation 2: Overfitting**  
[Is the train-val gap larger now?]

**Observation 3: Complexity**  
[Is the added complexity worth the improvement?]

---

## 5. Residual Diagnostics

### What to Look For:

1. **Patterns in residual plots** → Model is missing something (nonlinearity, interactions)
2. **Funnel shape** → Heteroskedasticity (variance changes with prediction)
3. **Non-normal residuals** → Outliers or wrong model family
4. **Systematic errors in ranges** → Model struggles in certain regions

In [ ]:
# Get predictions
y_pred_baseline = baseline_pipeline.predict(X_val)
y_pred_poly = poly_pipeline.predict(X_val)

residuals_baseline = y_val - y_pred_baseline
residuals_poly = y_val - y_pred_poly

# Residual comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Baseline residuals
axes[0, 0].scatter(y_pred_baseline, residuals_baseline, alpha=0.5)
axes[0, 0].axhline(y=0, color='r', linestyle='--')
axes[0, 0].set_xlabel('Predicted')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].set_title('Baseline Model Residuals')

# Polynomial residuals
axes[0, 1].scatter(y_pred_poly, residuals_poly, alpha=0.5, color='orange')
axes[0, 1].axhline(y=0, color='r', linestyle='--')
axes[0, 1].set_xlabel('Predicted')
axes[0, 1].set_ylabel('Residuals')
axes[0, 1].set_title('Polynomial Model Residuals')

# Baseline distribution
axes[1, 0].hist(residuals_baseline, bins=50, edgecolor='black', alpha=0.7)
axes[1, 0].axvline(x=0, color='r', linestyle='--')
axes[1, 0].set_xlabel('Residual')
axes[1, 0].set_title('Baseline Residual Distribution')

# Polynomial distribution
axes[1, 1].hist(residuals_poly, bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1, 1].axvline(x=0, color='r', linestyle='--')
axes[1, 1].set_xlabel('Residual')
axes[1, 1].set_title('Polynomial Residual Distribution')

plt.tight_layout()
plt.show()

print("=== RESIDUAL STATISTICS ===")
print(f"Baseline MAE: {mean_absolute_error(y_val, y_pred_baseline):.4f}")
print(f"Polynomial MAE: {mean_absolute_error(y_val, y_pred_poly):.4f}")
print(f"\nBaseline RMSE: {np.sqrt(mean_squared_error(y_val, y_pred_baseline)):.4f}")
print(f"Polynomial RMSE: {np.sqrt(mean_squared_error(y_val, y_pred_poly)):.4f}")

## 📝 PAUSE-AND-DO Exercise 2 (10 minutes)

**Task:** Write a short diagnostic conclusion (what error patterns suggest).

Look at the residual plots above and answer:
1. Do you see any patterns (non-random scatter)?
2. Is there a funnel shape (heteroskedasticity)?
3. Does the polynomial model fix any issues?
4. What would you try next?

---

### YOUR DIAGNOSTIC CONCLUSION:

**Pattern Analysis:**  
[Describe any patterns you see in residuals]

**Heteroskedasticity:**  
[Is variance constant across predictions?]

**Model Comparison:**  
[Which model has better residual behavior?]

**Next Steps:**  
[What would you try to improve the model?]

---

## 6. Model Comparison Table

In [ ]:
# Create comparison table
comparison = pd.DataFrame([
    {
        'Model': 'Baseline Linear',
        'Features': X_train.shape[1],
        'Train_R2': train_score,
        'Val_R2': val_score,
        'Overfit_Gap': train_score - val_score,
        'Val_MAE': mean_absolute_error(y_val, y_pred_baseline)
    },
    {
        'Model': 'With Interaction',
        'Features': X_train_interact.shape[1],
        'Train_R2': train_score_interact,
        'Val_R2': val_score_interact,
        'Overfit_Gap': train_score_interact - val_score_interact,
        'Val_MAE': mean_absolute_error(y_val, interact_pipeline.predict(X_val_interact))
    },
    {
        'Model': 'Polynomial (deg=2)',
        'Features': n_features_poly,
        'Train_R2': train_score_poly,
        'Val_R2': val_score_poly,
        'Overfit_Gap': train_score_poly - val_score_poly,
        'Val_MAE': mean_absolute_error(y_val, y_pred_poly)
    }
])

print("=== MODEL COMPARISON ===")
print(comparison.to_string(index=False))

# Best model
best_idx = comparison['Val_R2'].idxmax()
print(f"\n✓ Best validation R²: {comparison.loc[best_idx, 'Model']} ({comparison.loc[best_idx, 'Val_R2']:.4f})")

## 7. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Coefficient Interpretation**: Requires scaling, caution about multicollinearity
2. **Feature Engineering**: Interactions and polynomials can help, but watch overfitting
3. **Residual Diagnostics**: Visual patterns reveal model inadequacies
4. **Complexity Tradeoff**: More features ≠ better performance
5. **Validation Discipline**: Always check validation scores, not just training

### Critical Rules:

> **"Scale features before interpreting coefficients"**

> **"Polynomial features explode exponentially - use sparingly"**

> **"Residual plots never lie"**

### Next Steps:

- Next notebook: Regularization (Ridge/Lasso) + Project proposal
- Regularization will help control overfitting from complex features
- Start thinking about your project dataset and target

---

## Bibliography

- James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). *An Introduction to Statistical Learning with Python* - Linear Regression chapter
- Hastie, T., Tibshirani, R., & Friedman, J. (2009). *The Elements of Statistical Learning* - Linear methods
- scikit-learn User Guide: [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
- scikit-learn User Guide: [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

---



<center>

Thank you!

</center>